# Importing dependencies

In [ ]:
from pyfaidx import Fasta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from bedparse import bedline
import numpy as np
import json
from statistics import mean
import matplotlib.pyplot as plt
import matplotlib.style as style 
from pyensembl import Genome
from statsmodels.stats.multitest import multipletests
import os,re
from bedparse import bedline
from collections import Counter
%matplotlib inline

# Validating 
Fast5 raw data are publicly available in SRA under the accession [SRP174366](https://www.ncbi.nlm.nih.gov/sra/?term=SRP174366) 
The pre-processing were done like described in Material and Methods with the [reference sequence](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE124309) also found in the repository including the processed files from m6Anet.

### Loading m6Anet ouput files

In [ ]:
#IVT with m6A instead of adenosine 
curlcake_mod = pd.read_csv('./IVT/modified/data.result.csv')
#IVT unmodified 
curlcake_non_mod = pd.read_csv('./IVT/non_modified/data.result.csv')

### Initialize modified arrays

In [ ]:
#identified modifications within modified sequences 
curlcake_mod_05 = curlcake_mod[curlcake_mod['probability_modified']>0.5]
#identified unmodified positions within the unmodified sequence 
curlcake_non_mod_05 = curlcake_non_mod[curlcake_non_mod['probability_modified']>0.5]

### Adding kmer Information to the output file

In [ ]:
#loading datapreparation file from m6Anet
data_index_mod = pd.read_csv('./IVT/modified/data.index')    
read_count_mod = pd.read_csv('./IVT/modified/data.readcount')
data_info_mod = data_index_mod.merge(read_count_mod, on=['transcript_id', 'transcript_position'])
#filtering for transcripts with equal or more than 20 reads as m6Anet is doing in the analysis
data_info_mod = data_info_mod[data_info_mod["n_reads"] >= 20].reset_index(drop=True)

#function for loading 5-mer 
def _load_data_mod(idx):
    """[function for loading 5-mer from index]

    Args:
        idx ([int]): [index of the m6Anet output file]

    Returns:
        [string]: [5-mer of the index position]
    """  
    #reading json file and initialize variables 
    with open('./IVT/modified/data.json', 'r') as f:
        tx_id, tx_pos, start_pos, end_pos = data_info_mod.iloc[idx][["transcript_id", "transcript_position","start", "end"]]
        f.seek(start_pos, 0)
        #initialize json file with start end end postion
        json_str = f.read(end_pos - start_pos)
        #loading json information for certain id on certain position
        pos_info = json.loads(json_str)[tx_id][str(tx_pos)]
        #ensure that a certain position on the transcript only have a certain kmer 
        assert(len(pos_info.keys()) == 1)
        # separate kmer and other features from datapreparation step of m6Anet
        kmer, features = list(pos_info.items())[0]
    return kmer

###### SAME STEPS ARE DONE FOR THE NOT MODIFIED DATASET #######
 

data_index_nonmod = pd.read_csv('./IVT/not_modified/data.index')    
read_count_nonmod = pd.read_csv('./IVT/not_modified/data.readcount')
data_info_nonmod = data_index_nonmod.merge(read_count_nonmod, on=['transcript_id', 'transcript_position'])
data_info_nonmod = data_info_nonmod[data_info_nonmod["n_reads"] >= 20].reset_index(drop=True)


def _load_data_nonmod(idx):
    """[function for loading 5-mer from index]

    Args:
        idx ([int]): [index of the m6Anet output file]

    Returns:
        [string]: [5-mer of the index position]
    """  
    with open('./non_modified/m6anet_dataprep/data.json', 'r') as f:
        tx_id, tx_pos, start_pos, end_pos = data_info_nonmod.iloc[idx][["transcript_id", "transcript_position","start", "end"]]
        f.seek(start_pos, 0)
        json_str = f.read(end_pos - start_pos)
        pos_info = json.loads(json_str)[tx_id][str(tx_pos)]
        
        assert(len(pos_info.keys()) == 1)

        kmer, features = list(pos_info.items())[0]
    return kmer

### Adding the kmer informations to the m6Anet output file

In [ ]:
#iterating through the m6Anet output file and initialize the kmer column with the 5-mer's 
for index,row in curlcake_non_mod.iterrows():
    curlcake_non_mod.loc[index,'kmer']=_load_data_nonmod(index)[1:-1]
for index,row in curlcake_mod.iterrows():
    curlcake_mod.loc[index,'kmer']=_load_data_mod(index)[1:-1]

### Counting the ratio of modified 5-mers


In [ ]:
#Counting all 5-mers and storing in a dictionary {kmer:count}
letter_counts_all = Counter(list(curlcake_mod['kmer']))
#Initialize a DataFrame from dictionary of kmer and count
all_kmer = pd.DataFrame.from_dict(letter_counts_all, orient='index')
all_kmer.columns = ['count']
all_kmer.sort_values(by='count',inplace=True)
all_kmer['kmer'] = all_kmer.index
#Counting only predicted m6A positions from m6Anet
letter_counts_mod = Counter(list(curlcake_mod[curlcake_mod['probability_modified']>0.5]['kmer']))
mod_kmer = pd.DataFrame.from_dict(letter_counts_mod, orient='index')
mod_kmer.columns=['count_modified']
mod_kmer.sort_values(by = 'count_modified',inplace=True)
mod_kmer['kmer_mod'] = mod_kmer.index
# concat both DataFrames and replace NAs 5-mers and counts with 0 
count_list=pd.concat([all_kmer,mod_kmer],axis=1)
count_list.fillna(0,inplace=True)

# calculate the percentage of identified 5-mers 
count_list['%'] = ((count_list['count_modified'] / count_list['count']*100))
#sort DataFrame by probability of the 5mer is identified by m6Anet and the number of the certain 5-mer occurs in the sequence
count_list.sort_values(by=['%','count'],ascending=False,inplace=True)

### Plotting the probability of a 5-mer being identified by m6Anet

In [ ]:
sns.set_style('darkgrid')
plt.style.use('ggplot')
plt.figure(figsize=(10, 7), dpi=1200)
ax=sns.barplot(x='kmer',y='%', data=count_list)
ax.set_xticklabels(rotation=90, horizontalalignment='right',labels=count_list['kmer'])
plt.show()

### Filtering the m6Anet output file for consecutively m6A positions

In [ ]:
#initialize a pd.Series with True and False on index where the 5-mer contain a m6A on position -1 
double_A = curlcake_mod.kmer.str[1].str.contains('A')
#initialize a pd.Series with True and False on index where the 5-mer doesn't contain a m6A on position -1 
notdouble = ~double_A
#filter for position with and without m6A on position -1
without_consecutive_m6A_curlcake = curlcake_mod.query('@notdouble')
consecutive_m6a_curlcake = curlcake_mod.query('@double')

### Plotting the percentage ratio of a modified DRACH motif gets identified 

In [ ]:
#sns.set_style('darkgrid')
plt.style.use('ggplot')
plt.figure(figsize=(10, 7), dpi=200)
clrs = ['red' if x[1]=='A' else 'grey' for x in count_list.kmer ]
ax=sns.barplot(x='kmer',y='%', data=count_list, palette=clrs)
ax.set_xticklabels(rotation=90, horizontalalignment='right',labels=count_list['kmer'])
plt.ylabel('identified as modified [%]')
plt.ylim(-10,110)
plt.hlines(0,xmin=12.7,xmax=13.4)
plt.hlines(0,xmin=13.7,xmax=14.4)
plt.hlines(0,xmin=14.7,xmax=15.4)
plt.hlines(0,xmin=15.7,xmax=16.4)
plt.hlines(0,xmin=16.7,xmax=17.4)
red_patch = mpatches.Patch(color='red', label='consecutively_m6A_motif')
green_patch = mpatches.Patch(color='grey',label='not_consecutively_m6A_motif')
plt.legend(handles=[green_patch,red_patch,])
plt.show()

### Plotting the distribution of the probability_modified values  

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(10, 7), dpi=300)

sns.distplot(curlcake_mod.probability_modified,bins=30,kde=False,label='modified')
sns.distplot(curlcake_non_mod.probability_modified,bins=30,kde=False,label='non modified')
sns.distplot(without_consecutive_m6A_curlcake.probability_modified,bins=30,kde=False,label='modified_without_consecutively_m6A_motif')
sns.distplot(consecutive_m6a_curlcake.probability_modified,bins=30,kde=False,label='modified_with_consecutively_m6A_motif')
black_patch = mpatches.Patch(color='red', label='The red data')
plt.legend(loc='upper right')
plt.xlim(0,1)
plt.rcParams['font.family'] = "serif"
sns.set_context('paper') 
plt.xlabel('probability_modified')
plt.ylabel('count')
sns.set_style("dark")
plt.vlines(0.5,ymin=0,ymax=60,linestyle='--',color='black')
plt.show()

# Loading the reference genome file

downloaded from [ensemble](http://nov2020.archive.ensembl.org/Homo_sapiens/Info/Index#:~:text=cDNAs%2C%20ncRNA%2C%20proteins-,Download%20GTF,-or%20GFF3%20files)

In [ ]:
genome = Genome(reference_name='GRCh38',
                annotation_name='my_genome_features',
                gtf_path_or_url='./Homo_sapiens.GRCh38.102.gtf',
                transcript_fasta_paths_or_urls='./Homo_sapiens.GRCh38.102.fa')

# Loading the m6Anet output

In [ ]:
m6a_1 = pd.read_csv('./polysome_profiling/data.result_monosome.csv')
m6a_3 = pd.read_csv('./polysome_profiling/data.result_polysome.csv')

# Functions for analyzing the data


### adding transcriptomic information


In [ ]:
def create_tx_ref(transcript_ids):
    """[create pd.DataFrame with the transcript_ids and the respective postions of the regions and the chromosome]

    Args:
        transcript_ids ([numpy.ndarray]): [unique transcript_ids]

    Returns:
        [pd:DataFrame]: [transcript_id','end_5utr','end_cds','end_tx','chromosome']
    """    
    #initialize arrays
    transcript_lengths = []
    #looping through transcript_ids
    for tx_id in transcript_ids:
        # safe genome information from transcript_id in tx
        try:
            tx = genome.transcript_by_id(tx_id)
        except Exception:
            continue
        # end of transcript (end_tx) defined as length of sequence
        end_tx = len(tx.sequence)
        chromosome = tx.contig
        
        # including only transcripts containing start and stop codon
        if tx.contains_start_codon and tx.contains_stop_codon:
            end_5utr = len(tx.five_prime_utr_sequence)
            
            #end of CDS (end_cds) as the position of the last stop codon
            end_cds = tx.last_stop_codon_spliced_offset
            if end_tx > end_cds > end_5utr > 0:
                transcript_lengths += [(tx_id,end_5utr,end_cds,end_tx,chromosome)]
    return pd.DataFrame(transcript_lengths,columns= ['transcript_id','end_5utr','end_cds','end_tx','chromosome'])

### calculating the relative m6A positions on the transcript from the m6Anet output file

In [ ]:
def region_assign(df):
    """[calculating the relative positions of m6A]

    Args:
        df ([pd.DataFrame]): [m6Anet outputfile with transcript_ids and transcript_position]

    Returns:
        [pd.DataFrame]: [transcript_position assigned to the region and the respective relative position]
    """    
    #Add postions of the regions to df
    df = df.merge(create_tx_ref(df.transcript_id.unique()),how='inner',on='transcript_id')
    #Assign the postions in df to the region where it is located as boolean
    df['isin_5utr'] = df['transcript_position'] < df['end_5utr']
    df['isin_cds'] = (~df['isin_5utr']) & (df['transcript_position'] < df['end_cds'])
    df['isin_3utr'] = (~df['isin_5utr']) & (~df['isin_cds']) 
    #Calculate the length of the regions except the length of 5'UTR with is defined by the end of it in df['end_5utr']
    len_cds = df['end_cds']-df['end_5utr']
    len_3utr = df['end_tx']-df['end_cds']
    
    #ensure that those requirements are met
    assert (len_cds > 0).all()
    assert (len_3utr > 0).all()

    # calculate the relative length of the positions 
    rel_len_5utr = df['isin_5utr']*(df['transcript_position']/df['end_5utr'])
    rel_len_cds = df['isin_cds']*((df['transcript_position']-df['end_5utr'])/len_cds)
    rel_len_3utr = df['isin_3utr']*((df['transcript_position']-df['end_cds'])/len_3utr)
    
    df['rel_5utr'] = rel_len_5utr
    df['rel_cds'] = rel_len_cds
    df['rel_3utr'] = rel_len_3utr
    
    return df

### refering to create_tx_ref but creating a list of length of certain regions 

In [ ]:
def create_tx_ref_region_length(transcript_ids,region):
    """[creating a list of containing the lengths of regions of respective transcript_id]

    Args:
        transcript_ids ([numpy.ndarray]): [unique transcript_ids]
        region ([str]): [region of interest:['5utr','cds','3utr']]

    Returns:
        [list]: [list of lengths of regions of respective transcript_id ]
    """    
    #initialize array 
    transcript_lengths = []
    if region=='5utr':
        for tx_id in transcript_ids:
            try:
                tx = genome.transcript_by_id(tx_id)
            except Exception:
                continue
            five = len(tx.five_prime_utr_sequence)
            transcript_lengths.append(five)
    if region == 'cds':
        for tx_id in transcript_ids:
            try:
                tx = genome.transcript_by_id(tx_id)
            except Exception:
                continue
            end_5utr = len(tx.five_prime_utr_sequence)
            end_cds = tx.last_stop_codon_spliced_offset
            cds=end_cds-end_5utr
            transcript_lengths.append(cds)
    if region == '3utr':
        for tx_id in transcript_ids:
            try:
                tx = genome.transcript_by_id(tx_id)
            except Exception:
                continue
            end_cds = tx.last_stop_codon_spliced_offset
            three=len(tx.sequence)-end_cds
            transcript_lengths.append(three)

    return transcript_lengths

## Metagene plot
### plotting the relative positions on the transcriptome

In [ ]:
def plot_rel_positions(ax,df, label):
    """[transcriptome wide plotting of m6A positions]

    Args:
        ax ([ax]): [ax]
        df ([pd.DataFrame]): [DataFrame containing transcript_positions and transcript_id]
        label ([type]): [description]

    Returns:
        [type]: [description]
    """    


    #Add postions of the regions to df
    df = df.merge(create_tx_ref(df),how='inner',on='transcript_id')
    #Assign the postions in df to the region as boolean 
    df['isin_5utr'] = df['transcript_position'] < df['end_5utr']
    df['isin_cds'] = (~df['isin_5utr']) & (df['transcript_position'] < df['end_cds'])
    df['isin_3utr'] = (~df['isin_5utr']) & (~df['isin_cds']) 
    #Calculate the length of the regions except the length of 5'UTR with is defined by the end of it in df['end_5utr']
    len_cds = df['end_cds']-df['end_5utr']
    len_3utr = df['end_tx']-df['end_cds']
    
    #ensure that those requirements are met
    assert (len_cds > 0).all()
    assert (len_3utr > 0).all()

    #calculate the relative length of the positions 
    rel_len_5utr = df['isin_5utr']*(df['transcript_position']/df['end_5utr'])
    rel_len_cds = df['isin_cds']*((df['transcript_position']-df['end_5utr'])/len_cds)
    rel_len_3utr = df['isin_3utr']*((df['transcript_position']-df['end_cds'])/len_3utr)
    #Initialize the relative positions
    rel_positions = list(rel_len_5utr[rel_len_5utr>0])
    rel_positions += list(rel_len_cds[rel_len_cds>0] + 1)
    rel_positions += list(rel_len_3utr[rel_len_3utr>0] +2)
    #ploting the relative positions in a Kernel Density Estimate plot using Gaussian kernels
    sns.kdeplot(rel_positions, ax=ax, label=label, shade=True, cut=True)
    ax.set_xticks([])
    
    plt.yticks(fontsize=14)
    #annotating the x axis 
    trans = ax.get_xaxis_transform()
    ax.plot([-.05,1],[-.08,-.08], color="k", transform=trans, clip_on=False,linewidth=7.0)
    ax.annotate("5'UTR",xy=(0.5, -0.1),xytext=(0.5,-0.1),             
                annotation_clip=False,fontsize=20)
    ax.plot([1,2],[-.08,-.08], color="k", transform=trans, clip_on=False,linewidth=20.0)
    ax.annotate("CDS",xy=(1.5, -0.1),xytext=(1.5,-0.1),             
                annotation_clip=False,fontsize=20)
    ax.plot([2,3],[-.08,-.08], color="k", transform=trans, clip_on=False,linewidth=7.0)
    ax.annotate("3'UTR",xy=(2.5, -0.1),xytext=(2.5,-0.1),             
                annotation_clip=False,fontsize=20)
    ax.axvline(1, ymin=0, color='black', linestyle="--")
    ax.axvline(2, ymin=0, color='black', linestyle="--")
    ax.set_xlim([0,3])
    ax.set_ylabel('density',fontsize=16)
    return ax

### Count of modification per transcript and optional for region and calculating the respective modification status

In [ ]:
def count_mod(df,fraction,region='all'):
    """[count of modification per transcript and optional for region and calculating the respective modification status]

    Args:
        df ([pd.DataFrame]): [DataFrame containing transcript_positions and transcript_id]
        fraction ([str]): ['polysome','monosome']
        region (str, optional): [region of interest:['5utr','cds','3utr']]. Defaults to 'all'.

    Returns:
        [pd.DataFrame]: [DataFrame with number modification, length and modification of respective transcript_id]
    """    

    #calculating modification status of each transcript without region
    if region == 'all':
        df1=pd.concat([df.transcript_id.value_counts()],axis=1,keys=[fraction])
        df1['length']= create_tx_ref(list(df1.index))
        df1['mod_status'] = df1[fraction]/df1['length']
    #calculating modification status of each transcript with region information and regional modification status
    elif region != 'all':
        #counting transcript_ids with modifications in certain region
        df1=pd.concat([df[df['isin_'+str(region)]==True].transcript_id.value_counts()],axis=1,keys=[str(fraction + ' ' + region)])
        #initialize length of the transcript
        df1['length']= list(create_tx_ref(list(df1.index)).end_tx)
        #initialize modification status of the region
        df1['mod_status'] = df1[str(fraction+ ' ' + region)]/df1['length']
        #initialize the length of the region
        df1[region+'_length']= create_tx_ref_region_length(list(df1.index),region)
        df1['mod_status_'+region] = df1[str(fraction+ ' ' + region)]/df1[region+'_length']
    return df1

## Filtering steps of the m6Anet output

In [ ]:
# initialize the index as a column
m6a_1['index']= m6a_1.index
m6a_3['index']=m6a_3.index
# filter fractions for modified positions
m6a_3_mod=m6a_3[m6a_3['probability_modified']>0.5]
m6a_3_mod= region_assign(m6a_3_mod)

m6a_1_mod=m6a_1[m6a_1['probability_modified']>0.5]
m6a_1_mod= region_assign(m6a_1_mod)

#list of processed transcripts 
m6a_3_transcript_li=list(set(m6a_3.transcript_id))
m6a_1_transcript_li=list(set(m6a_1.transcript_id))
#list of transcripts with no modifications in fractions
m6a_3_transcript_zero = list(set(m6a_3[~m6a_3['transcript_id'].isin(list(set(m6a_3_mod.transcript_id)))]['transcript_id']))
m6a_1_transcript_zero = list(set(m6a_1[~m6a_1['transcript_id'].isin(list(set(m6a_1_mod.transcript_id)))]['transcript_id']))
#list of transcripts with modifications in fractions
m6a_3_transcript_mod=list(set(m6a_3_mod.transcript_id))
m6a_1_transcript_mod=list(set(m6a_1_mod.transcript_id))

#initialize differential modified positions
m6a_diff_poly= pd.merge(m6a_3, m6a_1, on=['transcript_id','transcript_position'],how='left')
m6a_diff_mono= pd.merge(m6a_1, m6a_3, on=['transcript_id','transcript_position'],how='left')

#positions modified in one fraction and unmodified in the other fraction
m6a_diff_poly_only2=m6a_diff_poly[(m6a_diff_poly['probability_modified_y']<0.5)&(m6a_diff_poly['probability_modified_x']>0.5)]
m6a_diff_mono_only2=m6a_diff_mono[(m6a_diff_mono['probability_modified_y']<0.5)&(m6a_diff_mono['probability_modified_x']>0.5)]
#initialize differential modified positions per fraction and assign the region information
m6a_3_diff_mod=m6a_diff_poly_only2[['transcript_id','transcript_position','n_reads_x', 'probability_modified_x']]
#m6a_3_diff_mod=region_assign(m6a_3_diff_mod)

m6a_1_diff_mod=m6a_diff_mono_only2[['transcript_id','transcript_position','n_reads_x', 'probability_modified_x']]
#m6a_1_diff_mod=region_assign(m6a_1_diff_mod)

#list of differential modified positions
m6a_1_diffmodpos_trans = list(set(m6a_1_diff_mod.transcript_id))
m6a_3_diffmodpos_trans = list(set(m6a_3_diff_mod.transcript_id))

#filter for transcripts in fraction which are differential modified in the other fraction
m6a_3_test_on_monosome = m6a_3_diff_mod[m6a_3_diff_mod['transcript_id'].isin(m6a_1_diffmodpos_trans)]
m6a_1_test_on_polysome = m6a_1_diff_mod[m6a_1_diff_mod['transcript_id'].isin(m6a_3_diffmodpos_trans)]
#filter for transcripts in fraction which are not differential modified in the other fraction
m6a_3_test_noton_monosome = m6a_3_diff_mod[~m6a_3_diff_mod['transcript_id'].isin(m6a_1_diffmodpos_trans)]
m6a_1_test_noton_polysome = m6a_1_diff_mod[~m6a_1_diff_mod['transcript_id'].isin(m6a_3_diffmodpos_trans)]

# Plotting the relation between the length and the number of modifications

In [ ]:
#initialize list of lengths
list_of_length1=[]
#initialize list of modificationnumber in a range of 1 to 30 because no transcript with more modifications then 29
list_of_mod1=list(range(1,30))
for elem in list_of_mod1:
    # Assigning the length of 0 for transcripts without a certain number of modifications
    if len(list(count_mod(m6a_1_mod,'monosome')\
                                [count_mod(m6a_1_mod,'monosome')['monosome']==elem]\
                                       ['length'].values))==0:
        list_of_length1.append(list(np.random.normal(0,0,100)))
    # Assinging the a list length of transcripts with certain modification number
    else:
        list_of_length1.append(list(count_mod(m6a_1_mod,'monosome')\
                                [count_mod(m6a_1_mod,'monosome')['monosome']==elem]['length'].values))
# initialize a dictionary with the number of modification and the respective lengths of the transcripts
m_dict = {key: value for key, value in zip(list_of_mod1, list_of_length1)}

###### SAME STEPS ARE DONE FOR THE POLYSOME FRACTION #######

list_of_length3=[]
list_of_mod3=list(range(1,30))
for elem in list_of_mod3:
    if len(list(count_mod(m6a_3_mod,'polysome')\
                                [count_mod(m6a_3_mod,'polysome')['polysome']==elem]\
                ['length'].values))==0:
        list_of_length3.append(list(np.random.normal(0,0,100)))
        
    else:
        list_of_length3.append(list(count_mod(m6a_3_mod,'polysome')\
                                 [count_mod(m6a_3_mod,'polysome')['polysome']==elem]['length'].values))
p_dict = {key: value for key, value in zip(list_of_mod3, list_of_length3)}

## plotting the length of the transcripts vs. the number of modifications

In [ ]:
plt.style.use('ggplot')
f, axs = plt.subplots(1,2,figsize=(30,8),sharey=True)
axs[0].boxplot(m_dict.values())
axs[0].set_xticklabels(m_dict.keys())
axs[0].set_title('length and n_mod. in monosome')
axs[1].boxplot(p_dict.values())
axs[1].set_xticklabels(p_dict.keys())
axs[1].set_title('length and n_mod in polysome')
for ax in axs.flatten():
    ax.xaxis.set_tick_params(labelbottom=True)
    ax.yaxis.set_tick_params(labelleft=True)

axs[0].set_xlabel('n_mod')
axs[1].set_xlabel('n_mod')
axs[0].set_ylabel('transcripts length')
axs[1].set_ylabel('transcripts length')

# Plotting the distributions of the modification status 

In [ ]:
plt.style.use('ggplot')
fig, axs = plt.subplots(1, figsize=(20, 12))
csfont = {'fontname':'Times New Roman'}
sns.distplot(\
             #count_mod(region_assign(m6a_3_diff_mod,'polysome','cds')['mod_status']\
             #count_mod(region_assign(m6a_3_diff_mod,'polysome')['mod_status']\
             #count_mod(m6a_3_mod,'polysome')['mod_status']\
             #count_mod(region_assign(m6a_3_test_on_monosome ),'polysome','cds')['mod_status']\
             #count_mod(region123_fast(m6a_diff_poly_only1_notin1),'polysome','3utr','3utr')['length'])\
             ,color='r',label='polysome',bins=50,kde=False,norm_hist=True,hist_kws= {'histtype':'stepfilled', 'alpha':0.3, 'ec':"k"})
sns.distplot(\
             #count_mod(m6a_1_mod,'monosome')['mod_status']\
             #count_mod(region_123_fast,'monosome','cds','3utr')['3utr_length']\
             #count_mod(region123_fast(m6a_1[m6a_1['probability_modified']<0.5]),'monosome')['length']\
             #count_mod(region123_fast(m6a_diff_mono_only1_notin3),'monosome','cds','3utr')['mod_status_cds']\
             #count_mod(region123_fast(m6a_diff_mono_only1_notin3),'monosome','3utr','cds')['length'])\
             ,color='b',label='monosome',bins=50,kde=False,norm_hist=True,hist_kws= {'histtype':'stepfilled', 'alpha':0.3,'density': True, 'ec':"k"})
plt.xlabel('mod_status_new',fontsize=16)
plt.ylabel('density',fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=16)

plt.show()